In [6]:
# CS350 Assignment 3: Fraud Detection Analysis for Second-Party Fraud using Neo4j and GDS
# Author: Team 7 for CS 350
# Date: November 2024

# Note to viewer/developer: some steps may be labeled "miscellaneously" or shifted for the purpose of program execution 

# Step 1: Connect to Neo4j Server and Initialize GDS
# -----------------------------------------------------
from neo4j import GraphDatabase
from neo4j.exceptions import Neo4jError
import pandas as pd
import os

# Define Neo4j connection parameters
NEO4J_URI = "bolt://localhost:7687"  
NEO4J_USER = "neo4j"                  # Username to the test database
NEO4J_PASSWORD = "79327932"            # Password to the test database

# Initialize the Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Function to execute a Cypher query and return all records as a list of dictionaries
def run_cypher_query(query, parameters=None):
    with driver.session() as session:
        try:
            result = session.run(query, parameters)
            records = result.data()
            return records
        except Neo4jError as e:
            print(f"Neo4jError executing query: {e}")
            return None
        except Exception as e:
            print(f"Unexpected error: {e}")
            return None

print("Connected to Neo4j successfully.")

# Step 2: Check Neo4j Version. Likewise, verify APOC and GDS plugins installation
# ---------------------------------------------------
print("\n--- Step 1: Check Neo4j Version ---")
check_neo4j_version_query = """
CALL dbms.components() YIELD name, versions, edition
RETURN name, versions, edition
"""

neo4j_version_record = run_cypher_query(check_neo4j_version_query)
if neo4j_version_record:
    for component in neo4j_version_record:
        name = component.get('name', 'Unknown')
        versions = component.get('versions', ['Unknown'])
        edition = component.get('edition', 'Unknown')
        print(f"Component: {name}")
        print(f"Versions: {versions}")
        print(f"Edition: {edition}\n")
else:
    print("Failed to retrieve Neo4j version.")

with driver.session() as session:
    # Check if APOC is installed
    apoc_installed = session.run("""
        RETURN apoc.version() AS apocVersion
    """).single()
    
    # Check if GDS is installed
    gds_installed = session.run("""
        RETURN gds.version() AS gdsVersion
    """).single()
    
    apoc_version = apoc_installed["apocVersion"] if apoc_installed else "APOC not installed"
    gds_version = gds_installed["gdsVersion"] if gds_installed else "GDS not installed"
    
    print(f"APOC Version: {apoc_version}")
    print(f"GDS Version: {gds_version}")



# Function to drop existing graph projections to prevent conflicts
def drop_graph_projection(graph_name):
    drop_query = f"""
    CALL gds.graph.exists('{graph_name}') YIELD exists
    CALL apoc.do.when(
        exists, 
        'CALL gds.graph.drop("{graph_name}") YIELD graphName RETURN graphName', 
        '', 
        {{}}  // Empty map for 'apoc.do.when'
    ) YIELD value
    RETURN value
    """
    
    result = run_cypher_query(drop_query)
    if result:
        print(f"Projection '{graph_name}' dropped if it existed.")
    else:
        print(f"Failed to drop projection '{graph_name}' or it did not exist.")

# Step 3: Create SHARED_PII Relationships and Visualize Schema
print("\n--- Step 3: Create SHARED_PII Relationships ---")
create_shared_pii_query = """
MATCH (c1:Client)-[:HAS_EMAIL|HAS_PHONE|HAS_SSN]->(p)<-[:HAS_EMAIL|HAS_PHONE|HAS_SSN]-(c2:Client)
WHERE c1.id < c2.id
MERGE (c1)-[:SHARED_PII]->(c2)
"""

shared_pii_result = run_cypher_query(create_shared_pii_query)
if shared_pii_result is not None:
    print("SHARED_PII relationships created successfully.")
else:
    print("Failed to create SHARED_PII relationships.")

# Verify the schema visualization 
print("\n--- Verify Schema Visualization ---")
schema_visualization_query = "CALL db.schema.visualization() YIELD nodes, relationships RETURN nodes, relationships"

schema_records = run_cypher_query(schema_visualization_query)
if schema_records:
    print("Schema visualization executed. Please use Neo4j Browser to view the visual schema.")
else:
    print("Failed to visualize schema.")

# Drop existing projections if necessary
drop_graph_projection('clientClusters')
drop_graph_projection('similarity')

# Step 4: Create In-Memory Projection 'clientClusters'
# ---------------------------------------------------
print("\n--- Step 4: Create In-Memory Projection 'clientClusters' ---")
create_projection_query = """
CALL gds.graph.project(
    'clientClusters',
    'Client',
    'SHARED_PII'
)
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount
"""

projection_records = run_cypher_query(create_projection_query)
if projection_records:
    projection_result = projection_records[0]
    print(f"Projection created: {projection_result['graphName']}")
    print(f"Nodes in projection: {projection_result['nodeCount']}")
    print(f"Relationships in projection: {projection_result['relationshipCount']}")
else:
    print("Failed to create in-memory projection. It might already exist or there might be issues with the data.")

# Verify the projection list
print("\n--- Verify In-Memory Projections ---")
list_graphs_query = "CALL gds.graph.list() YIELD graphName RETURN graphName"

graphs_records = run_cypher_query(list_graphs_query)
if graphs_records:
    print("Current in-memory projections:")
    for graph in graphs_records:
        print(f"- {graph['graphName']}")
else:
    print("Failed to list in-memory projections.")

# Step 5: Identify Clusters Using Weakly Connected Components (WCC)
# ---------------------------------------------------
print("\n--- Step 5: Identify Clusters Using WCC ---")
wcc_query = """
CALL gds.wcc.stream('clientClusters')
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).id AS clientId,
       componentId AS clusterId
"""

wcc_records = run_cypher_query(wcc_query)
if wcc_records:
    if len(wcc_records) > 0:
        wcc_df = pd.DataFrame(wcc_records)
        print("Clusters identified using WCC algorithm:")
        display(wcc_df.head())
    else:
        print("No clusters identified.")
else:
    print("Failed to run WCC algorithm.")

# Step 6: Mark Clients in Clusters as Potential Fraud Rings
# ---------------------------------------------------
print("\n--- Step 6: Mark Clients in Clusters as Potential Fraud Rings ---")
mark_fraud_rings_query = """
CALL gds.wcc.stream('clientClusters')
YIELD nodeId, componentId
WITH gds.util.asNode(nodeId) AS client, componentId AS clusterId
WITH clusterId, collect(client.id) AS clients
WITH clusterId, clients, size(clients) AS clusterSize
WHERE clusterSize >= 2
UNWIND clients AS clientId
MATCH (c:Client) WHERE c.id = clientId
SET c.SecondPartyFraudster = true
"""

mark_fraud_rings_result = run_cypher_query(mark_fraud_rings_query)
if mark_fraud_rings_result is not None:
    print("Clients in clusters of size >= 2 have been marked with 'SecondPartyFraudster'.")
else:
    print("Failed to mark clients with 'SecondPartyFraudster'.")

# Step 7: Create Bipartite Graph Using Cypher Projection
# ---------------------------------------------------
print("\n--- Step 7: Create Bipartite Graph Using Cypher Projection ---")
create_similarity_graph_query = """
CALL gds.graph.project(
    'similarity',
    ['Client', 'Email', 'Phone', 'SSN'],
    ['HAS_EMAIL', 'HAS_PHONE', 'HAS_SSN']
)
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount
"""

similarity_graph_records = run_cypher_query(create_similarity_graph_query)
if similarity_graph_records:
    similarity_graph_result = similarity_graph_records[0]
    print(f"Similarity graph created: {similarity_graph_result['graphName']}")
    print(f"Nodes in similarity graph: {similarity_graph_result['nodeCount']}")
    print(f"Relationships in similarity graph: {similarity_graph_result['relationshipCount']}")
else:
    print("Failed to create similarity graph. It might already exist or there might be issues with the data.")

# Step 8: Compute Similarity Scores Using nodeSimilarity Algorithm
# ---------------------------------------------------
print("\n--- Step 8: Compute Similarity Scores Using nodeSimilarity Algorithm ---")
node_similarity_query = """
CALL gds.nodeSimilarity.mutate(
    'similarity',
    {
        mutateProperty: 'jaccardScore',
        mutateRelationshipType: 'SIMILAR_TO',
        topK: 15
    }
)
YIELD nodesCompared, relationshipsWritten, similarityDistribution
RETURN nodesCompared, relationshipsWritten, similarityDistribution
"""

node_similarity_records = run_cypher_query(node_similarity_query)
if node_similarity_records:
    node_similarity_result = node_similarity_records[0]
    print(f"nodeSimilarity algorithm executed.")
    print(f"Nodes compared: {node_similarity_result['nodesCompared']}")
    print(f"Relationships written: {node_similarity_result['relationshipsWritten']}")
else:
    print("Failed to run nodeSimilarity algorithm.")

# Step 9: Write 'SIMILAR_TO' Relationships and Compute Degree Centrality
# ---------------------------------------------------
print("\n--- Step 9: Write 'SIMILAR_TO' Relationships and Compute Degree Centrality ---")
compute_degree_query = """
CALL gds.degree.write(
    'similarity',
    {
        nodeLabels: ['Client'],
        relationshipTypes: ['SIMILAR_TO'],
        relationshipWeightProperty: 'jaccardScore',
        writeProperty: 'secondPartyFraudScore'
    }
)
YIELD nodePropertiesWritten
RETURN nodePropertiesWritten
"""

degree_records = run_cypher_query(compute_degree_query)
if degree_records:
    degree_result = degree_records[0]
    node_properties_written = degree_result.get('nodePropertiesWritten', 'N/A')
    print(f"Degree centrality computed and 'secondPartyFraudScore' set.")
    print(f"Nodes with 'secondPartyFraudScore': {node_properties_written}")
    # If 'maxDegree' is needed and available, it can be printed conditionally
    if 'maxDegree' in degree_result:
        print(f"Maximum degree: {degree_result['maxDegree']}")
    else:
        print("Maximum degree information not available in the current GDS version.")
else:
    print("Failed to compute degree centrality.")

# Step 10: Label High Centrality Clients as Potential Fraudsters
# ---------------------------------------------------
print("\n--- Step 10: Label High Centrality Clients as Potential Fraudsters ---")
    
# Implementing IQR-Based Thresholding with Enhanced Handling
iqr_threshold_query = """
MATCH (c:Client)
WHERE c.secondPartyFraudScore IS NOT NULL
WITH 
    percentileCont(c.secondPartyFraudScore, 0.25) AS Q1,
    percentileCont(c.secondPartyFraudScore, 0.75) AS Q3
WITH 
    Q1, Q3, (Q3 - Q1) AS IQR
WITH 
    Q1, Q3, IQR, 
    CASE 
        WHEN IQR > 0 THEN (Q3 + 1.5 * IQR) 
        ELSE Q3 
    END AS upperThreshold
RETURN Q1, Q3, IQR, upperThreshold
"""

iqr_stats = run_cypher_query(iqr_threshold_query)
if iqr_stats:
    Q1 = iqr_stats[0].get('Q1', None)
    Q3 = iqr_stats[0].get('Q3', None)
    IQR = iqr_stats[0].get('IQR', None)
    upperThreshold = iqr_stats[0].get('upperThreshold', None)
    
    print(f"Calculated Quartiles and IQR:")
    print(f"Q1 (25th percentile): {Q1}")
    print(f"Q3 (75th percentile): {Q3}")
    print(f"IQR (Q3 - Q1): {IQR}")
    print(f"Upper Threshold (Q3 + 1.5 * IQR): {upperThreshold}")
    
    if upperThreshold and isinstance(upperThreshold, (int, float)):
        label_fraudsters_query = """
        WITH $upperThreshold AS threshold
        MATCH (c:Client)
        WHERE c.secondPartyFraudScore > threshold
        SET c.SecondPartyFraudster = true
        RETURN COUNT(DISTINCT c) AS fraudsterCount
        """

        parameters = {'upperThreshold': upperThreshold}
        
        label_fraudsters_records = run_cypher_query(label_fraudsters_query, parameters)
        
        fraudster_count = label_fraudsters_records[0].get('fraudsterCount', 0)
        threshold = label_fraudsters_records[0].get('threshold', 'N/A')
        if fraudster_count > 0:
            print(f"Clients labeled as 'SecondPartyFraudster': {fraudster_count}")
            print(f"Threshold for labeling: {threshold}")
        else:
            print("No clients exceeded the IQR-based upper threshold for labeling as fraudsters.")

percentile = 0.95 
# Execute labeling query with the determined percentile
label_fraudsters_query = f"""
MATCH (c:Client)
WHERE c.secondPartyFraudScore IS NOT NULL
WITH percentileCont(c.secondPartyFraudScore, {percentile}) AS threshold
MATCH (c:Client)
WHERE c.secondPartyFraudScore > threshold
SET c.SecondPartyFraudster = true
RETURN count(c) AS fraudsterCount, threshold
"""

label_fraudsters_records = run_cypher_query(label_fraudsters_query)
if label_fraudsters_records:
    label_result = label_fraudsters_records[0]
    fraudster_count = label_result.get('fraudsterCount', 0)
    threshold = label_result.get('threshold', 'N/A')
    if fraudster_count > 0:
        print(f"Clients labeled as 'SecondPartyFraudster': {fraudster_count}")
        print(f"Threshold for labeling: {threshold}")
    else:
        print("No clients exceeded the threshold for labeling as fraudsters.")
else:
    print("Failed to label fraudsters.")

# Step 11: List Potential Fraudsters
# ---------------------------------------------------
print("\n--- Step 11: List Potential Fraudsters ---")
list_fraudsters_query = """
MATCH (c:Client)
WHERE c.SecondPartyFraudster = true
RETURN c.id AS ClientID, c.name AS Name
"""

fraudsters_records = run_cypher_query(list_fraudsters_query)
if fraudsters_records:
    if len(fraudsters_records) > 0:
        fraudsters_df = pd.DataFrame(fraudsters_records)
        print("Potential Second-Party Fraudsters:")
        display(fraudsters_df)
    else:
        print("No potential fraudsters found.")
else:
    print("Failed to retrieve fraudsters.")

# Step 12: Cleanup - Close Neo4j Driver
# ---------------------------------------------------
print("\n--- Step 12: Cleanup - Close Neo4j Driver ---")
try:
    driver.close()
    print("Neo4j connection closed.")
except Exception as e:
    print(f"Error closing Neo4j driver: {e}")

Connected to Neo4j successfully.

--- Step 1: Check Neo4j Version ---
Component: Neo4j Kernel
Versions: ['5.25.1']
Edition: enterprise

APOC Version: 5.25.1
GDS Version: 2.12.0

--- Step 3: Create SHARED_PII Relationships ---
SHARED_PII relationships created successfully.

--- Verify Schema Visualization ---
Schema visualization executed. Please use Neo4j Browser to view the visual schema.
Projection 'clientClusters' dropped if it existed.
Projection 'similarity' dropped if it existed.

--- Step 4: Create In-Memory Projection 'clientClusters' ---
Projection created: clientClusters
Nodes in projection: 10
Relationships in projection: 12

--- Verify In-Memory Projections ---
Current in-memory projections:
- clientClusters

--- Step 5: Identify Clusters Using WCC ---
Clusters identified using WCC algorithm:


,clientId,clusterId
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0



--- Step 6: Mark Clients in Clusters as Potential Fraud Rings ---
Clients in clusters of size >= 2 have been marked with 'SecondPartyFraudster'.

--- Step 7: Create Bipartite Graph Using Cypher Projection ---
Similarity graph created: similarity
Nodes in similarity graph: 43
Relationships in similarity graph: 40

--- Step 8: Compute Similarity Scores Using nodeSimilarity Algorithm ---
nodeSimilarity algorithm executed.
Nodes compared: 10
Relationships written: 24

--- Step 9: Write 'SIMILAR_TO' Relationships and Compute Degree Centrality ---
Degree centrality computed and 'secondPartyFraudScore' set.
Nodes with 'secondPartyFraudScore': 10
Maximum degree information not available in the current GDS version.

--- Step 10: Label High Centrality Clients as Potential Fraudsters ---
Calculated Quartiles and IQR:
Q1 (25th percentile): 0.12698412698412698
Q3 (75th percentile): 0.5555555555555556
IQR (Q3 - Q1): 0.4285714285714286
Upper Threshold (Q3 + 1.5 * IQR): 1.1984126984126986
No clients 

,ClientID,Name
0,1,Alice
1,2,Bob
2,3,Charlie
3,4,Diana
4,5,Eve
5,6,Frank
6,7,Grace



--- Step 12: Cleanup - Close Neo4j Driver ---
Neo4j connection closed.
